In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
data = pd.read_csv("../Data/MAGIC.csv")

In [45]:
data

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


### Probleme mit dem Datenset

Zuerst müssen wir das Datenset modifizieren, damit es für ML verwendet werden kann

- Alle Werte numerisch machen
- Werte skalieren (Outlier reduzieren)
- Unebenheiten ausbessern (Neue Daten erzeugen, die die Unebenheiten ausgleichen)

In [46]:
data["class"].unique()

array(['g', 'h'], dtype=object)

In [47]:
data["class"] == 'g'

0         True
1         True
2         True
3         True
4         True
         ...  
19015    False
19016    False
19017    False
19018    False
19019    False
Name: class, Length: 19020, dtype: bool

In [48]:
data["class"] = (data["class"] == 'g').astype(int)

In [49]:
data

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,0
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,0
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,0
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,0


### Aufteilen der Daten in Trainings- und Testsets

Um ein Modell trainieren zu können benötigen wir ein Trainingsset und ein Testset

Dafür können wir das originale Datenset in zwei Teile aufteilen: 80% Training, 20% Test

In [50]:
# Schritt 1: Datenset in 80/20 aufteilen
# data.sample(frac=1)  # Gibt das Datenset in einer zufälligen Ordnung zurück

sampled = data.sample(frac=1)
training = sampled[:int(len(data)*0.8)]  # 80%
test = sampled[int(len(data)*0.8):]  # 20%

# Schritt 2: class Spalte von den restlichen Daten trennen
left_train = training.drop(columns = ["class"])
right_train = training["class"]

left_test = test.drop(columns = ["class"])
right_test = test["class"]

In [51]:
len(test) + len(training)

19020

### Werte skalieren

Hier werden jetzt Outlier reduziert und alle Werte auf eine ähnliche Höhe gebracht

Statt 6-Stelligen Werten ist jetzt jeder Wert einstellig

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(left_train)

left_train = pd.DataFrame(scaler.fit_transform(left_train))

In [53]:
left_train

,0,1,2,3,4,5,6,7,8,9
0,2.751695,0.847017,0.759400,-0.931459,-0.772477,3.830944,-2.405811,-1.282901,0.152148,-0.039021
1,-0.417297,-0.222989,0.224834,-0.242731,-0.279314,0.527706,0.318810,0.400287,-0.766218,-0.494767
2,0.995092,0.722767,1.996021,-1.103505,-1.131059,0.306929,0.808233,-1.104830,-0.856471,2.124603
3,-0.675984,-0.471860,-0.832471,0.873653,0.769335,0.504350,0.086069,-0.242859,-0.871513,-1.186168
4,1.675941,3.747538,1.913016,-1.581409,-1.597125,0.093199,-0.907578,3.379010,0.587646,-1.081434
...,...,...,...,...,...,...,...,...,...,...
15211,0.890704,0.960929,2.697652,-1.531707,-1.486931,-0.716471,1.448469,1.114555,-1.031966,1.612045
15212,-0.624186,-0.198715,-0.388513,-0.287517,-0.293765,0.228871,-0.662745,-0.533082,-0.613832,0.061418
15213,1.976196,3.601316,0.718637,-0.882849,-0.850155,-3.415057,2.287788,4.079228,-0.348782,1.653013
15214,-0.461303,-0.610703,-0.041710,-0.035184,-0.173636,0.438473,0.254923,0.222984,0.537095,-0.808051


### Unebenheiten ausbessern

Momentan gibt es noch eine unterschiedliche Anzahl von 0 und 1 Daten

Diese müssen gleichmäßig sein, damit das Model keinen Bias entwickelt

In [54]:
right_train.value_counts()

class
1    9836
0    5380
Name: count, dtype: int64

In [58]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
left, right = ros.fit_resample(left_train, right_train)

left_train = left
right_train = right

right.value_counts()

class
0    9836
1    9836
Name: count, dtype: int64

## Verschiedene vorgegebene Algorithmen/Modelle

### k-nearest neighbors (kNN)

- Neuer Datenpunkt wird platziert

- Anhand von k Nachbarn wird dieser Datenpunkt klassifiziert

- Von der Klasse von der es mehr Nachbarn gibt, wird dieser Datenpunkt eingeteilt

- k muss ungerade sein (3, 5, 7, 9)

In [59]:
from sklearn.neighbors import KNeighborsClassifier

In [60]:
knnModel = KNeighborsClassifier(7)  # Anzahl Nachbarn

In [61]:
knnModel.fit(left_train, right_train.values.reshape(-1))  # Hier soll für die Rechte Seite ein 1D-Array übergeben werden

KNeighborsClassifier(n_neighbors=7)

In [62]:
prediction = knnModel.predict(left_test)

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [63]:
pd.DataFrame(np.hstack((left_test, right_test.values.reshape(len(prediction), 1), prediction.reshape(len(prediction), 1))), columns=[*range(10), "Actual", "Prediction"])

,0,1,2,3,4,5,6,7,8,9,Actual,Prediction
0,27.2257,11.0172,2.6196,0.5522,0.3469,15.7050,23.8955,-8.1105,49.7560,212.7540,0.0,0.0
1,78.4363,23.4979,3.5105,0.2473,0.1736,-52.3365,43.3518,-12.1062,0.2600,288.8380,1.0,0.0
2,24.5405,15.2548,2.5740,0.3920,0.2040,9.8866,12.2024,13.4030,18.7850,122.4800,1.0,0.0
3,48.9080,23.1113,2.4807,0.3537,0.1769,-55.9717,37.9526,13.6168,2.5147,214.1940,1.0,0.0
4,36.4896,14.4627,2.8280,0.3492,0.1924,7.2028,22.4112,6.3171,4.7061,223.7650,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3799,78.4068,54.3684,3.4650,0.1429,0.0620,11.1058,40.3272,30.5371,63.3984,278.5333,0.0,0.0
3800,21.0167,8.4981,2.3541,0.6770,0.4137,22.0759,13.7412,-0.9055,9.3744,180.5790,1.0,0.0
3801,38.0349,12.8489,2.2504,0.5730,0.3062,30.0231,17.8761,5.8187,40.2139,246.0870,0.0,0.0
3802,20.7843,15.7948,2.5899,0.4910,0.3021,24.2409,11.9971,7.5164,16.0930,222.3740,1.0,0.0


In [64]:
comparison = pd.Series(right_test.values == prediction).value_counts()
comparison

False    2496
True     1308
Name: count, dtype: int64

In [65]:
print(comparison[True] / len(right_test))
print(comparison[False] / len(right_test))

0.3438485804416404
0.6561514195583596


Funktion zum Evaluieren des Models

In [66]:
def evaluate(prediction):
    comparison = pd.Series(right_test.values == prediction).value_counts()
    print(f"Richtige Vorhersage: {round(comparison[True] / len(right_test) * 100, 2)}%")
    print(f"Falsche Vorhersage: {round(comparison[False] / len(right_test) * 100, 2)}%")

In [67]:
evaluate(prediction)

Richtige Vorhersage: 34.38%
Falsche Vorhersage: 65.62%


### Naive Bayes

Berechnet für jede Spalte eine Wahrscheinlichkeit in Relation zum Gesamtdatenset

Danach wird jeder dieser Wahrscheinlichkeiten mit 50% verglichen

Im Anschluss werden die Wahrscheinlichkeiten zusammengeführt, und wieder mit 50% verglichen

In [68]:
from sklearn.naive_bayes import GaussianNB

In [69]:
nb = GaussianNB()

In [70]:
nb.fit(left_train, right_train.values.reshape(-1))

GaussianNB()

In [71]:
prediction = nb.predict(left_test)

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


In [72]:
evaluate(prediction)

Richtige Vorhersage: 34.38%
Falsche Vorhersage: 65.62%


### Logistische Regression

Ähnlich wie Naive Bayes, aber fasst einen Datensatz zu einer Wahrscheinlichkeit zusammen und prüft dann, ob dieser über einem Schwellwert liegt

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
lr = LogisticRegression()

In [36]:
lr.fit(left_train, right_train.values.reshape(-1))

LogisticRegression()

In [37]:
prediction = lr.predict(left_test)

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [38]:
evaluate(prediction)

Richtige Vorhersage: 35.86%
Falsche Vorhersage: 64.14%


### Support Vector Machines

Legt eine Hyperplane durch das Datenset durch

Die Hyperplane breitet sich aus, alle Punkte die getroffen werden, werden klassifiziert

In 2D noch visualisierbar, unser Datenset hat 14 Dimensionen

Sehr auswändiger Algorithmus, aber dafür eher präzise

In [39]:
from sklearn.svm import SVC

In [40]:
svc = SVC()
svc.fit(left_train, right_train.values.reshape(-1))

SVC()

In [41]:
prediction = svc.predict(left_test)

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [42]:
evaluate(prediction)

Richtige Vorhersage: 35.86%
Falsche Vorhersage: 64.14%


## Neurales Netzwerk

Das Neurale Netzwerk besteht aus Layern (Schichten)

Jede Schicht enthält beliebig viele Neuronen (Nodes)

Jedes Neuron summiert seine Inputs auf, und schickt diese Summe in die gegebene Activation Function

- Activation Function: Normale Mathematische Funktion, welche am Ende des Summenprozesses ausgeführt wird, und genau einen Wert als Ergebnis bringt

Der Output der Activation Function wird an den nächsten Layer weitergegeben

---

Wir können jetzt unser eigenes Modell aufbauen mithilfe von **Keras**

Was ist ein Modell?

Ein Modell ist ein Programm, welches per Machine Learning erzeugt, anstatt per Hand geschrieben wird

Das Model nimmt im Anschluss als Input einen Datensatz und gibt ein Ergebnis zurück

---

Loss: Beschreibt, wie gut das Model im klassifizieren von Daten ist

Dense: Ein Layer, welche alle Nodes mit allen nächsten Nodes verbindet

Dropout: Ein Layer, welcher X% der Inputdaten wegwirft. Wird verwendet, um das Model zu verlangsamen

In [74]:
import setuptools.dist
import tensorflow as tf

In [82]:
model = tf.keras.Sequential([
    tf.keras.Input((10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [83]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),  # Optimizer: Beeinflusst die Lernrate (hier 0.001)
              loss='binary_crossentropy',  # Loss-Funktion: Hier Binary Crossentropy (0 oder 1)
              metrics=['accuracy'])

In [85]:
history = model.fit(
    left_train,
    right_train.values.reshape(-1),
    epochs=20,  # Anzahl Durchläufe
    batch_size=8,  # Parallelsierung
    verbose=1)  # Output aktivieren

Epoch 1/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8261 - loss: 0.3863
Epoch 2/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8362 - loss: 0.3666
Epoch 3/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8424 - loss: 0.3588
Epoch 4/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8459 - loss: 0.3528
Epoch 5/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8443 - loss: 0.3480
Epoch 6/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8466 - loss: 0.3410
Epoch 7/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8452 - loss: 0.3410
Epoch 8/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8522 - loss: 0.3388
Epoch 9/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8436 - loss: 0.3442
Epoch 10/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8501 - loss: 0.3393
Epoch 11/20
2459/2459 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8544 - loss: 0.3265
Epoch 12/20
2459/2459 ━━━━━━━

In [87]:
prediction = model.predict(left_test)

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [88]:
prediction

array([[1.9678064e-35],
       [3.5698145e-24],
       [5.6102937e-20],
       ...,
       [2.1781943e-37],
       [1.3338698e-26],
       [2.2020637e-14]], dtype=float32)

In [89]:
evaluate(prediction.reshape(-1))

Richtige Vorhersage: 17.93%
Falsche Vorhersage: 82.07%


In [91]:
model.save("../Models/Income.keras")

In [93]:
m = tf.keras.models.load_model("../Models/Income.keras")

In [96]:
evaluate((model.predict(left_test) < 0.5).astype(int).reshape(-1))

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Richtige Vorhersage: 65.62%
Falsche Vorhersage: 34.38%


In [86]:
def trainModel(neuronen, dropout, learningRate, batch):
    model = tf.keras.Sequential([
        tf.keras.Input((10,)),
        tf.keras.layers.Dense(neuronen, activation='relu'),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(neuronen, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learningRate),  # Optimizer: Beeinflusst die Lernrate (hier 0.001)
              loss='binary_crossentropy',  # Loss-Funktion: Hier Binary Crossentropy (0 oder 1)
              metrics=['accuracy'])
    
    history = model.fit(
        left_train,
        right_train.values.reshape(-1),
        epochs=20,  # Anzahl Durchläufe
        batch_size=batch,  # Parallelsierung
        verbose=1)  # Output aktivieren

    return model

In [ ]:
best = 0
bestModel = None
for anzNeuronen in [16, 32, 64]:
    for dropout in [0, 0.2]:
        for learningRate in [0.001, 0.005, 0.01]:
            for batch in [32, 64]:
                model = trainModel(anzNeuronen, dropout, learningRate, batch)
                prediction = model.predict(left_test)
                equal = right_test == (model.predict(left_test) < 0.5).astype(int)
                anzRichtig = equal.astype(int) == 1
                if anzRichtig > best:
                    best = anzRichtig
                    bestModel = model
bestModel.save("../Models/Best.keras")